## Doing function calling with open source local models like llama are experimental in langchain till now.
## https://api.python.langchain.com/en/latest/llms/langchain_experimental.llms.ollama_functions.OllamaFunctions.html#langchain_experimental.llms.ollama_functions.OllamaFunctions
## Video to explain function calling in LLama models: https://www.youtube.com/watch?v=Ss_GdU0KqE0

## Experiment with Llama and Phi3 models for function calling: https://export.arxiv.org/pdf/2404.14219

In [40]:
from langchain_community.chat_models import ChatOllama
from langchain_core.messages import HumanMessage
from langchain_core.output_parsers import StrOutputParser, JsonOutputParser, PydanticOutputParser
from langchain_core.prompts import ChatPromptTemplate, PromptTemplate

In [33]:
from langchain_core.pydantic_v1 import BaseModel, Field

In [38]:
from langchain_experimental.llms.ollama_functions import OllamaFunctions

In [27]:
import json

In [2]:
import langchain

In [3]:
langchain.debug=True

In [4]:
local_llm = ChatOllama(model="llama2:latest", keep_alive=-1, temperature=0, max_new_tokens=512)

## Part1: Basic usgae of local models like llama2

In [5]:
## This is assumed to be given by the human
prompt = ChatPromptTemplate.from_template("Write me a 500 word article on {topic} from the perspective of a {profession}.")
chain = prompt | local_llm |  StrOutputParser()

In [6]:
output = chain.invoke({"topic": "LLMs", "profession": "baker"})

[chain/start] [chain:RunnableSequence] Entering Chain run with input:
{
  "topic": "LLMs",
  "profession": "baker"
}
[chain/start] [chain:RunnableSequence > prompt:ChatPromptTemplate] Entering Prompt run with input:
{
  "topic": "LLMs",
  "profession": "baker"
}
[chain/end] [chain:RunnableSequence > prompt:ChatPromptTemplate] s] Exiting Prompt run with output:
[outputs]
[llm/start] [chain:RunnableSequence > llm:ChatOllama] Entering LLM run with input:
{
  "prompts": [
    "Human: Write me a 500 word article on LLMs from the perspective of a baker."
  ]
}
[llm/end] [chain:RunnableSequence > llm:ChatOllama] [18.91s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": "\nAs a baker, I can't help but think about the importance of mastering the art of bread-making. It's not just about mixing flour, water, yeast, and salt to create a delicious loaf - it's about understanding the science behind the process. And that's where LLMs come in.\n\nLLMs, or Large Language 

In [7]:
print(output)


As a baker, I can't help but think about the importance of mastering the art of bread-making. It's not just about mixing flour, water, yeast, and salt to create a delicious loaf - it's about understanding the science behind the process. And that's where LLMs come in.

LLMs, or Large Language Models, are AI systems designed to process and generate human-like language. They have the potential to revolutionize the way we work, communicate, and create. But what does this have to do with bread-making? Well, let me tell you.

As a baker, I know that the key to creating the perfect loaf is not just about following a recipe, but understanding the underlying principles of bread-making. It's about knowing how yeast works, how dough develops, and how different ingredients interact with each other. And that's where LLMs come in.

LLMs can help us understand these underlying principles by analyzing and generating text related to bread-making. They can provide insights into the chemical reactions t

## Streaming output

In [9]:
for chunk in chain.stream({"topic": "LLMs", "profession": "deep sea diver"}):
    print(chunk, end="", flush=True)

[chain/start] [chain:RunnableSequence] Entering Chain run with input:
{
  "input": ""
}
[chain/start] [chain:RunnableSequence > prompt:ChatPromptTemplate] Entering Prompt run with input:
{
  "topic": "LLMs",
  "profession": "deep sea diver"
}
[chain/end] [chain:RunnableSequence > prompt:ChatPromptTemplate] s] Exiting Prompt run with output:
[outputs]
[llm/start] [chain:RunnableSequence > llm:ChatOllama] Entering LLM run with input:
{
  "prompts": [
    "Human: Write me a 500 word article on LLMs from the perspective of a deep sea diver."
  ]
}
[chain/start] [chain:RunnableSequence > parser:StrOutputParser] Entering Parser run with input:
{
  "input": ""
}

As a deep sea diver, I have had the privilege of exploring some of the most incredible and mysterious environments on Earth. From the vibrant coral reefs of the Caribbean to the icy waters of the Arctic, I have seen it all. But one thing that has always fascinated me is the world of LLMs – Large Marine Life.

For those who may not be

## Part2: Define json schema that the LLM output should follow.
## Very important when creating agents

In [20]:
json_schema = {
    "title": "Person",
    "description": "Identifying information about a person",
    "type": "object",
    "properties": {
        "name": {"title": "Name", "description": "The person's name", "type": "string"},
        "age": {"title": "Age", "description": "The person's age", "type": "integer"},
        "city": {"title": "City", "description": "The person's city", "type": "string"},
        "fav_food": {
            "title": "Fav Food",
            "description": "The person's favourite food",
            "type": "string"
        }
    },
    "required": ["name", "age"]
}

In [21]:
## Keep format as json that increase the chances that the llm is outputing json. It is not guaranteed
json_local_llm = ChatOllama(model="llama2:latest", format="json", keep_alive=-1, temperature=0.1, max_new_tokens=512)

In [22]:
## The messages will be used to create a prompt template
json_messages = [
    HumanMessage(content="Please tell me about the person using the following JSON schema:"),
    HumanMessage(content="{schema}"),
    HumanMessage(content="Considering the above mentioned schema, tell me about a person named Oliver who lives in Toronto, aged 32 and likes Biryani")
]

In [25]:
json_based_prompt = ChatPromptTemplate.from_messages(json_messages)
print(json_based_prompt)

input_variables=[] messages=[HumanMessage(content='Please tell me about the person using the following JSON schema:'), HumanMessage(content='{schema}'), HumanMessage(content='Considering the above mentioned schema, tell me about a person named Oliver who lives in Toronto, aged 32 and likes Biryani')]


json_dumps = json.dumps(json_schema, indent=2)

In [30]:
json_chain = json_based_prompt | json_local_llm |  JsonOutputParser()
json_response = json_chain.invoke({"schema": json_dumps})

[chain/start] [chain:RunnableSequence] Entering Chain run with input:
{
  "schema": "{\n  \"title\": \"Person\",\n  \"description\": \"Identifying information about a person\",\n  \"type\": \"object\",\n  \"properties\": {\n    \"name\": {\n      \"title\": \"Name\",\n      \"description\": \"The person's name\",\n      \"type\": \"string\"\n    },\n    \"age\": {\n      \"title\": \"Age\",\n      \"description\": \"The person's age\",\n      \"type\": \"integer\"\n    },\n    \"city\": {\n      \"title\": \"City\",\n      \"description\": \"The person's city\",\n      \"type\": \"string\"\n    },\n    \"fav_food\": {\n      \"title\": \"Fav Food\",\n      \"description\": \"The person's favourite food\",\n      \"type\": \"string\"\n    }\n  },\n  \"required\": [\n    \"name\",\n    \"age\"\n  ]\n}"
}
[chain/start] [chain:RunnableSequence > prompt:ChatPromptTemplate] Entering Prompt run with input:
{
  "schema": "{\n  \"title\": \"Person\",\n  \"description\": \"Identifying informatio

In [78]:
print(type(json_response))

<class 'dict'>


In [79]:
print(json_response)

{'name': 'Oliver', 'age': 32, 'city': 'Toronto', 'interests': ['Biryani']}


## Part 3: Pydantic data model that the LLM output should follow

In [55]:
## Define Data model
class Person(BaseModel):
    name: str = Field(description="The person's name", required=True)
    age: int = Field(description="The person's age", required=True)
    city: str = Field(description="The person's city", required=True)

In [63]:
person_parser = PydanticOutputParser(pydantic_object=Person)

In [64]:
print(person_parser.get_format_instructions())

The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}
the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.

Here is the output schema:
```
{"properties": {"name": {"title": "Name", "description": "The person's name", "required": true, "type": "string"}, "age": {"title": "Age", "description": "The person's age", "required": true, "type": "integer"}, "city": {"title": "City", "description": "The person's city", "required": true, "type": "string"}}, "required": ["name", "age", "city"]}
```


In [ ]:
# data_model_prompt = ChatPromptTemplate.from_template("""You are a smart assistant who takes the following context and question and returns answer in JSON\n Context: {context} \n Question: {question}\n """)

# data_model_prompt.invoke({"question": "Extract information for person described in context", "context": "I know someone named Oliver who lives in Toronto, aged 32 and likes Biryani"})

In [70]:
data_model_prompt = PromptTemplate(
    template="Answer the user query with the following instructions and given context.\n{format_instructions}\n Context: {context} \n Question: {question}\n",
    input_variables=["query", "context"],
    partial_variables={"format_instructions": person_parser.get_format_instructions()},
)

In [90]:
experimental_local_llm = OllamaFunctions(model="llama2:latest", format="json", temperature=0)#.with_structured_output(Joke)
experimental_person_structured_local_llm = experimental_local_llm.with_structured_output(Person)
experiental_chain = data_model_prompt | experimental_person_structured_local_llm |  StrOutputParser()

In [91]:
experiental_response = experiental_chain.invoke({"question": "Extract information for person described in context", "context": "I know someone named Oliver who lives in Toronto, aged 32 and likes Biryani"})

[chain/start] [chain:RunnableSequence] Entering Chain run with input:
{
  "question": "Extract information for person described in context",
  "context": "I know someone named Oliver who lives in Toronto, aged 32 and likes Biryani"
}
[chain/start] [chain:RunnableSequence > prompt:PromptTemplate] Entering Prompt run with input:
{
  "question": "Extract information for person described in context",
  "context": "I know someone named Oliver who lives in Toronto, aged 32 and likes Biryani"
}
[chain/end] [chain:RunnableSequence > prompt:PromptTemplate] s] Exiting Prompt run with output:
[outputs]
[llm/start] [chain:RunnableSequence > llm:OllamaFunctions] Entering LLM run with input:
{
  "prompts": [
    "Human: Answer the user query with the following instructions and given context.\nThe output should be formatted as a JSON instance that conforms to the JSON schema below.\n\nAs an example, for the schema {\"properties\": {\"foo\": {\"title\": \"Foo\", \"description\": \"a list of strings\",

ValidationError: 1 validation error for Generation
text
  str type expected (type=type_error.str)

In [92]:
print(experiental_response)

name='Oliver' age=32 city='Toronto'


In [93]:
## Interesting: https://stackoverflow.com/questions/78591465/unexpected-string-validation-error-in-langchain-pydantic-output-parser
Person.parse_obj(experiental_response)

Person(name='Oliver', age=32, city='Toronto')

In [94]:
class Joke(BaseModel):
    setup: str = Field(description="question to setup a joke")
    punchline: str = Field(description="answer to resolve the joke")

joke_query = "Tell me a joke"
joke_parser = PydanticOutputParser(pydantic_object=Joke)
joke_prompt = PromptTemplate(template = "Answer the user query. \n{format_instructions}\n{query}\n", input_variables=["query"], partial_variables={"format_instructions": joke_parser.get_format_instructions()})

joke_chain = joke_prompt | structured_local_llm 

In [95]:
joke_prompt.invoke({"query": joke_query})

[chain/start] [prompt:PromptTemplate] Entering Prompt run with input:
{
  "query": "Tell me a joke"
}
[chain/end] [prompt:PromptTemplate] s] Exiting Prompt run with output:
[outputs]


StringPromptValue(text='Answer the user query. \nThe output should be formatted as a JSON instance that conforms to the JSON schema below.\n\nAs an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}\nthe object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.\n\nHere is the output schema:\n```\n{"properties": {"setup": {"title": "Setup", "description": "question to setup a joke", "type": "string"}, "punchline": {"title": "Punchline", "description": "answer to resolve the joke", "type": "string"}}, "required": ["setup", "punchline"]}\n```\nTell me a joke\n')

In [97]:
Joke.parse_obj(joke_chain.invoke({"query": joke_query}))

[chain/start] [chain:RunnableSequence] Entering Chain run with input:
{
  "query": "Tell me a joke"
}
[chain/start] [chain:RunnableSequence > prompt:PromptTemplate] Entering Prompt run with input:
{
  "query": "Tell me a joke"
}
[chain/end] [chain:RunnableSequence > prompt:PromptTemplate] s] Exiting Prompt run with output:
[outputs]
[llm/start] [chain:RunnableSequence > llm:OllamaFunctions] Entering LLM run with input:
{
  "prompts": [
    "Human: Answer the user query. \nThe output should be formatted as a JSON instance that conforms to the JSON schema below.\n\nAs an example, for the schema {\"properties\": {\"foo\": {\"title\": \"Foo\", \"description\": \"a list of strings\", \"type\": \"array\", \"items\": {\"type\": \"string\"}}}, \"required\": [\"foo\"]}\nthe object {\"foo\": [\"bar\", \"baz\"]} is a well-formatted instance of the schema. The object {\"properties\": {\"foo\": [\"bar\", \"baz\"]}} is not well-formatted.\n\nHere is the output schema:\n```\n{\"properties\": {\"setup

Joke(setup='Why was the math book sad? Because it had too many problems.', punchline='Get it? Problems!')